# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('argo', case=False)] 
    print(len(df))
    df.to_csv('../dataset/argo_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

13


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/argo_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

13


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['Compilation of Elyra-Pipelines to Tekton based Kubeflow fails-Ive installed '
 'a kubernetes cluster running kubeflow pipelines based on tekton on top of '
 'KIND using the following instructions Now Im getting the following error '
 'message from the Elyra pipelines editor. Running against an argo based kfp '
 'cluster works fine. Is the kfp compiler somehow not supporting tekton? Can '
 'someone please shine some light on this? HTTP response body: '
 '{&quot;error_message&quot;:&quot;Error creating pipeline: Create pipeline '
 'failed: Failed to get parameters from the pipelineRun: Invalid input error: '
 'Unsupported argo version. Expected: tekton.dev/v1beta1. Received: '
 'argoproj.io/v1alpha1&quot;, &quot;error_details&quot;:&quot;Error creating '
 'pipeline: Create pipeline failed: Failed to get parameters from the '
 'pipelineRun: Invalid input error: Unsupported argo version. Expected: '
 'tekton.dev/v1beta1. Received: argoproj.io/v1alpha1&quot;} ',
 'Accessing argo workflow 

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['compilation', 'of', 'elyra', 'pipelines', 'to', 'tekton', 'based', 'kubeflow', 'fails', 'ive', 'installed', 'kubernetes', 'cluster', 'running', 'kubeflow', 'pipelines', 'based', 'on', 'tekton', 'on', 'top', 'of', 'kind', 'using', 'the', 'following', 'instructions', 'now', 'im', 'getting', 'the', 'following', 'error', 'message', 'from', 'the', 'elyra', 'pipelines', 'editor', 'running', 'against', 'an', 'argo', 'based', 'kfp', 'cluster', 'works', 'fine', 'is', 'the', 'kfp', 'compiler', 'somehow', 'not', 'supporting', 'tekton', 'can', 'someone', 'please', 'shine', 'some', 'light', 'on', 'this', 'http', 'response', 'body', 'quot', 'error_message', 'quot', 'quot', 'error', 'creating', 'pipeline', 'create', 'pipeline', 'failed', 'failed', 'to', 'get', 'parameters', 'from', 'the', 'pipelinerun', 'invalid', 'input', 'error', 'unsupported', 'argo', 'version', 'expected', 'tekton', 'dev', 'beta', 'received', 'argoproj', 'io', 'alpha', 'quot', 'quot', 'error_details', 'quot', 'quot', 'error', 

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['compilation', 'base', 'fail', 've', 'instal', 'kubernete', 'cluster', 'run', 'kubeflow', 'pipeline', 'base', 'top', 'kind', 'use', 'follow', 'instruction', 'm', 'get', 'follow', 'error', 'message', 'pipeline', 'editor', 'run', 'argo', 'base', 'work', 'fine', 'compiler', 'somehow', 'support', 'tekton', 'shine', 'light', 'http', 'response', 'body', 'error', 'create', 'pipeline', 'create', 'pipeline', 'fail', 'fail', 'get', 'parameter', 'pipelinerun', 'invalid', 'input', 'error', 'unsupported', 'expect', 'beta', 'receive', 'alpha', 'error', 'create', 'pipeline', 'create', 'pipeline', 'fail', 'fail', 'get', 'parameter', 'pipelinerun', 'invalid', 'input', 'error', 'unsupported', 'expect', 'beta', 'receive', 'alpha', 'quot']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 3), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 4), (9, 1), (10, 5), (11, 2), (12, 5), (13, 1), (14, 2), (15, 3), (16, 1), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 6), (29, 2), (30, 1), (31, 2), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alpha', 2),
  ('argo', 1),
  ('base', 3),
  ('beta', 2),
  ('body', 1),
  ('cluster', 1),
  ('compilation', 1),
  ('compiler', 1),
  ('create', 4),
  ('editor', 1),
  ('error', 5),
  ('expect', 2),
  ('fail', 5),
  ('fine', 1),
  ('follow', 2),
  ('get', 3),
  ('http', 1),
  ('input', 2),
  ('instal', 1),
  ('instruction', 1),
  ('invalid', 2),
  ('kind', 1),
  ('kubeflow', 1),
  ('kubernete', 1),
  ('light', 1),
  ('m', 1),
  ('message', 1),
  ('parameter', 2),
  ('pipeline', 6),
  ('pipelinerun', 2),
  ('quot', 1),
  ('receive', 2),
  ('response', 1),
  ('run', 2),
  ('shine', 1),
  ('somehow', 1),
  ('support', 1),
  ('tekton', 1),
  ('top', 1),
  ('unsupported', 2),
  ('use', 1),
  ('ve', 1),
  ('work', 1)]]

In [10]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [11]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

100%|██████████| 540/540 [09:45<00:00,  1.08s/it]


In [12]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
297,100% Corpus,2,asymmetric,0.61,0.544267
299,100% Corpus,2,asymmetric,symmetric,0.544267
298,100% Corpus,2,asymmetric,0.9099999999999999,0.542629
411,100% Corpus,6,symmetric,0.31,0.537383
391,100% Corpus,6,0.01,0.31,0.537121
...,...,...,...,...,...
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.434032
423,100% Corpus,7,0.01,0.9099999999999999,0.431641
453,100% Corpus,8,0.01,0.9099999999999999,0.425680
473,100% Corpus,8,symmetric,0.9099999999999999,0.422407


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -5.540419487449212



Coherence Score:  0.46996447574133565


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.097358  0.012179       1        1  47.099079
2      0.002989 -0.049871       2        1  26.937860
3     -0.032738  0.019198       3        1  14.459541
5     -0.024466  0.001321       4        1   5.828460
4     -0.027079  0.009080       5        1   5.019328
1     -0.016064  0.008092       6        1   0.655733, topic_info=           Term       Freq      Total Category  logprob  loglift
77     workflow  26.000000  26.000000  Default  30.0000  30.0000
12         fail   9.000000   9.000000  Default  29.0000  29.0000
212      resume   3.000000   3.000000  Default  28.0000  28.0000
10        error  10.000000  10.000000  Default  27.0000  27.0000
216     suspend   2.000000   2.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
199       value   0.023395  13.773316   Topic6  -5.9381  -1.3508
33          run   0.023385   6.601594   Topic6  -5.9386  -0.6158
42         work   0.023379   6.480777   Topic6  -5.9388  -0.5976
15          get   0.023373   8.861082   Topic6  -5.9391  -0.9107
124  entrypoint   0.023363   7.313923   Topic6  -5.9395  -0.7192

[334 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
43        1  0.368760    access
43        3  0.368760    access
293       1  0.367178   account
293       2  0.367178   account
0         1  0.366519     alpha
...     ...       ...       ...
77        5  0.037676  workflow
160       1  0.228804      yaml
160       2  0.457608      yaml
292       1  0.358439       yml
292       3  0.358439       yml

[373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 6, 5, 2])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.046*"name" + 0.033*"template" + 0.024*"parameter" + 0.021*"workflow" + '
  '0.019*"value" + 0.018*"input" + 0.017*"argo" + 0.017*"artifact" + '
  '0.016*"spec" + 0.015*"event"'),
 (1,
  '0.003*"name" + 0.003*"template" + 0.003*"error" + 0.003*"artifact" + '
  '0.003*"workflow" + 0.003*"script" + 0.003*"input" + 0.003*"argo" + '
  '0.003*"use" + 0.003*"quot"'),
 (2,
  '0.023*"name" + 0.022*"template" + 0.019*"argo" + 0.015*"workflow" + '
  '0.012*"get" + 0.012*"string" + 0.012*"response" + 0.011*"body" + '
  '0.011*"error" + 0.011*"request"'),
 (3,
  '0.031*"workflow" + 0.016*"argo" + 0.013*"use" + 0.012*"run" + 0.012*"event" '
  '+ 0.011*"access" + 0.011*"kubectl" + 0.009*"create" + 0.009*"role" + '
  '0.009*"permission"'),
 (4,
  '0.021*"resume" + 0.019*"workflow" + 0.019*"suspend" + 0.014*"state" + '
  '0.009*"already" + 0.009*"phase" + 0.007*"use" + 0.007*"however" + '
  '0.007*"go" + 0.007*"status"'),
 (5,
  '0.016*"pipeline" + 0.014*"error" + 0.014*"fail" + 0.012*"create

In [12]:
num_topics = 6
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicArgo.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  65832799  Compilation of Elyra-Pipelines to Tekton based...   
1  66178222  Accessing argo workflow archive via http leads...   
2  69021388    AMQP - Argo-events: argo-workflow not triggered   
3  69094527                Why doesn't this argo workflow run?   
4  70495730  How to access a content of file which is passe...   

                                                Body  RatingsSentiCR  \
0  I've installed a kubernetes cluster running ku...              -1   
1  I'm trying to access the Argo workflow archive...              -1   
2  I am trying to run an argo-workflow triggered ...              -1   
3  I am trying to configure my Argo workflows.\nI...              -1   
4  I am trying to access the content(json data) o...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         